In [1]:
from time import sleep
import pandas as pandas
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
import pandas as pd
pd.set_option('display.max_rows', 200)


In [2]:
driver = webdriver.Chrome('/Users/admin/chromedriver')

In [96]:
def get_df_from_url(url, restaurant_name):
    driver.get(url)
    sleep(5)

    detailed_reviews_list=[]

    restaurant_header = driver.find_elements_by_css_selector('.photoHeader__373c0__YdvQE ')
    restaurant_average_rating = restaurant_header[0].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label').split()[0]
    restaurant_reviews_count = restaurant_header[0].find_element_by_css_selector('.text__373c0__2Kxyz').text.split()[0]
    try:
            exp = restaurant_header[0].find_elements_by_css_selector('.text__373c0__2Kxyz.text-bullet--after__373c0__3fS1Z.text-size--large__373c0__3t60B')[-1].text
            restaurant_expensiveness = len(exp)
    except:
            restaurant_expensiveness = 'N/C'

    while True:
        comment_blocs = driver.find_elements_by_css_selector('.review__373c0__13kpL.border-color--default__373c0__3-ifU')   
        
        for i in range(len(comment_blocs)):
            try:
                date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[3].text.split()[-1]
            except: 
                date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[2].text.split()[-1]
            rating = comment_blocs[i].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label')[0]

            try: 
                username = comment_blocs[i].find_elements_by_css_selector('.user-passport-info .link-size--inherit__373c0__1VFlE')[0].text
            except:
                username = 'Not_Yelp_User'
            
            try:
                photos_for_review = comment_blocs[i].find_elements_by_css_selector('a.link__373c0__1G70M.link-size--inherit__373c0__1VFlE')[1].text[0]
            except:
                photos_for_review = 0
            
            try:
                user_location =  comment_blocs[i].find_elements_by_css_selector('.user-passport-info .text__373c0__2Kxyz')[1].text
            except:
                user_location = 'no_location'

            text_review = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz .raw__373c0__3rcx7')[0].text
            user_friends_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[0].text
            user_reviews_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[1].text

            try:
                user_total_image_posted = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[2].text
                                                                                    
            except:
                user_total_image_posted = 0

            is_real_review = 1
            url = url

            detailed_reviews_list.append([date, username, photos_for_review, rating, text_review, user_location, user_friends_count, user_reviews_count,                                  user_total_image_posted, restaurant_average_rating, restaurant_reviews_count, restaurant_expensiveness, restaurant_name, url, is_real_review])

            
            print(url, i, 'OK', )

        try:
            #navigate to the next page
            driver.find_elements_by_css_selector('.next-link')[0].click()
            sleep(3) 
        except:
            break

    df = pd.DataFrame(detailed_reviews_list,
                                    columns = ['date','username', 'photos_for_review', 'rating',
                                                'text_review', 'user_location', 'user_friends_count',
                                                'user_reviews_count', 'user_total_image_posted', 'restaurant_average_rating',
                                                'restaurant_reviews_count', 'restaurant_expensiveness', 'restaurant_name', 'url', 'is_real_review'])
    return df


In [97]:
get_df_from_url('https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris', 'le drapeau' )

https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 0 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 1 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 2 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 3 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 4 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 5 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 6 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 7 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 8 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 9 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 10 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 11 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 12 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 13 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 14 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelité-paris 15 OK
https://www.yelp.f

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review
0,15/04/2016,Paul-Hector M.,0,5,Le meilleur bar/restaurant pour étudiants de P...,"Courbevoie, Hauts-de-Seine",0,7,0,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
1,09/11/2014,Clémentine R.,2,5,"Pour être honnête, j'ai longtemps hésité à rév...",Paris,275,459,976,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
2,01/03/2015,Fred L.,1,4,"Hors du temps, atypique .\n\nMr Quan nous prop...","Écully, Rhône",42,121,73,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
3,18/01/2018,Christine C.,0,3,"NOURRITURE\nPas mauvais, mais très loin des pl...",no_location,12,49,71,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
4,01/02/2016,Justine D.,0,2,Je suis venue deux fois au Drapeau de la Fidél...,"Invalides, Paris",19,22,0,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
5,30/04/2016,Marie B.,0,2,"Ok, c'est pas cher. Et encore. 2,5 pour un dem...",Paris,18,10,9,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
6,15/09/2015,Tony M.,0,5,"Manger et boire aussi bien, et pour si peu che...","Etterbeek, Belgique",0,43,0,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
7,21/02/2014,Eponine I.,0,4,Le drapeau de la fidélité fait partie de ces a...,Paris,2,14,0,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
8,12/10/2015,Hélène O.,0,2,"Testé à emporter, donc je ne juge que mon Bobu...",Paris,15,9,5,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1
9,20/08/2013,Sarah B.,0,4,J'avais entendu parler de ce restaurant il y a...,Paris,15,66,4,4,43,1,le drapeau,https://www.yelp.fr/biz/le-drapeau-de-la-fidel...,1


In [74]:
urls = ['https://www.yelp.fr/biz/le-ruisseau-paris-2',
        'https://www.yelp.fr/biz/maison-dong-paris']

names = ['le ruisseau', 'maison_dong']

In [496]:
df_list = [get_df_from_url(url, name) for url, name in zip(urls,names)]

https://www.yelp.fr/biz/le-ruisseau-paris-2 0 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 1 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 2 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 3 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 4 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 5 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 6 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 7 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 8 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 9 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 10 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 11 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 12 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 13 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 14 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 15 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 16 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 17 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 18 OK
https://www.yelp.fr/biz/le-ruisseau-paris-2 19 OK
https://ww

In [4]:
dataset = pd.read_csv('Yelp_Paris_list_of_restaurants/restaurants_dataset_only_with_reviews.csv', index_col=0)


In [98]:

df_list = [pd.DataFrame() for i in range(len(dataset))]
error_index_list = []
start = 398

for i in range(start, len(dataset)):
    try:
        df_list[i] = get_df_from_url(dataset['link_to_restaurant'][i], dataset['restaurant_name'][i])
    except:
        error_index_list.append((i, dataset['link_to_restaurant'][i], dataset['restaurant_name'][i]))
        print('restaurant number', i, 'seems to have a problem')
    print('restaurant number ', i, 'OK')

    if i % 50==0 and i != 0 :
        checkpoint = [df_list[k] for k in range(i-100+1, i+1)]
        pd.concat(checkpoint).to_csv('checkpoints/reviews_part'+ str(i) +'.csv')
        pd.DataFrame(error_index_list).to_csv('logfiles/logfile'+ str(i) +'.csv')
        print('checkpoint', i, 'successfully saved')


https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 0 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 1 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 2 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 3 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 4 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 5 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 6 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 7 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 8 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 9 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 10 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 11 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 12 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 13 OK
https://www.yelp.fr/biz/le-drapeau-de-la-fidelit%C3%A9-paris 14 OK
https

In [20]:
pd.read_csv('checkpoints/reviews_part20.csv')

,Unnamed: 0,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review
0,0,27/07/2017,Aimée M.,0,5,Restaurant un peu petit mais les plats sont de...,"Milan, Italie",4,1,0,4.0,5,2,Le Djoliba,https://www.yelp.fr/biz/le-djoliba-paris,1
1,1,14/04/2019,Johan E.,0,5,Excellent! J'étais de retour à Paris pour les ...,"Mechanicsburg, États-Unis",0,17,1,4.0,5,2,Le Djoliba,https://www.yelp.fr/biz/le-djoliba-paris,1
2,2,20/07/2015,Greg D.,0,4,Testé un midi du mois de juillet.\nA retenir e...,Paris,31,164,29,4.0,5,2,Le Djoliba,https://www.yelp.fr/biz/le-djoliba-paris,1
3,3,10/09/2016,Charles S.,0,2,"Bonne bouffe.\nHélas, d'énormes soucis d'organ...",Paris,0,1,0,4.0,5,2,Le Djoliba,https://www.yelp.fr/biz/le-djoliba-paris,1
4,0,20/01/2018,Keilon L.,8,4,Le resto c'est très simple et j'aime ça. C'es...,"Austin, États-Unis",311,378,2543,4.5,4,14,Chez Zeyna,https://www.yelp.fr/biz/chez-zeyna-paris,1
5,0,27/10/2015,Audrey C.,0,3,A la recherche d'un endroit où manger avant d'...,Lyon,41,66,153,4.0,6,10,New Soul Food,https://www.yelp.fr/biz/new-soul-food-paris,1
6,1,12/01/2020,Romeo C.,3,4,Je connaissais le food truck. Très bon concept...,Paris,2,30,63,4.0,6,10,New Soul Food,https://www.yelp.fr/biz/new-soul-food-paris,1
7,2,12/12/2017,Stephane K.,0,4,Un des meilleurs food truck de Paris. Devant l...,Villeurbanne,18,1,0,4.0,6,10,New Soul Food,https://www.yelp.fr/biz/new-soul-food-paris,1
8,3,12/06/2016,Peggy D.,0,4,Food truck sur le parvis Mk2 bibliothèque ambi...,"Montrouge, Hauts-de-Seine",11,394,246,4.0,6,10,New Soul Food,https://www.yelp.fr/biz/new-soul-food-paris,1
9,0,17/09/2020,Eva M.,0,5,Très bon plats et l'assiette avec différents a...,"Saint-Thomas, Haute-Garonne",0,2,1,4.0,17,2,Afrik’N’Fusion 20ème,https://www.yelp.fr/biz/afrik-n-fusion-20%C3%A...,1


In [513]:
dataset.sort_values('review_count', ascending=False)

,restaurant_name,link_to_restaurant,category,review_count
2328,L’As du Fallafel,https://www.yelp.fr/biz/l-as-du-fallafel-paris,Falafel,1810
1269,Angelina,https://www.yelp.fr/biz/angelina-paris,Petit déjeuner & brunch,1338
2569,Le Comptoir de la Gastronomie,https://www.yelp.fr/biz/le-comptoir-de-la-gast...,Français,1095
2572,Bouillon Chartier,https://www.yelp.fr/biz/bouillon-chartier-paris,Français,949
6194,L’Avant Comptoir,https://www.yelp.fr/biz/l-avant-comptoir-paris-3,Bars à tapas,613
...,...,...,...,...
5637,Aux Pains Perdus,https://www.yelp.fr/biz/aux-pains-perdus-paris-2,Sandwichs,1
2453,Mondial Chicken,https://www.yelp.fr/biz/mondial-chicken-paris,Fast Food,1
5634,Kazdal,https://www.yelp.fr/biz/kazdal-paris,Sandwichs,1
2455,Tell mom,https://www.yelp.fr/biz/tell-mom-paris,Fast Food,1
